# Export Data Exercise Answer Key

In [ ]:
import boto3
import time
import urllib3

from bosdyn.scout.client import ScoutClient

# We do not have an SSL Certificate, so ignore these warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

HOSTNAME = '10.202.101.3'
USERNAME = 'student'
PASSWORD = 'studentuser123'

MISSION = 'ThermalMissionExportData'
CHANNEL = 'thermal-inspection_advanced-anomaly_isotherm_image'
WALL = 'WallSideHeater'
ROOM = 'RoomSideHeater'
DAY = time.strftime('%Y%m%d')
BUCKET_NAME = 'export-data-training-bucket'

scout_client = ScoutClient(hostname=HOSTNAME, verify=False)
scout_client.authenticate_with_password(username=USERNAME, password=PASSWORD)

print('Scout IP:', scout_client._hostname)

In [ ]:
get_robots_response = scout_client.get_robots()

if get_robots_response.ok:
    all_robots = get_robots_response.json()
    for robot in all_robots:
        robot_hostname = robot.pop('hostname', None)
        if robot_hostname:
            print(robot_hostname)
else:
    print('get_robots() failed')
    print(get_robots_response.text)

### Task 1

In [ ]:
get_runs_response = scout_client.get_runs()

if get_runs_response.ok:
    runs_json = get_runs_response.json()
    runs = runs_json.get('resources') 
    for run in runs:
        print(run)
else:
    print('get_runs() failed')
    print(get_runs_response.text)

### Task 2

In [ ]:
get_runs_response = scout_client.get_runs(params={'missions': MISSION})

if get_runs_response.ok:
    get_runs_json = get_runs_response.json()
    runs = get_runs_json.get('resources')
    for run in runs:
        print(run)
else:
    print('get_runs() failed')
    print(get_runs_response.text)

### Task 3

In [ ]:
print(WALL)

wall_heater_params = {
    'missionName': MISSION,
    'actionName': WALL,
    'channelName': CHANNEL
}
get_run_captures_response = scout_client.get_run_captures(params=wall_heater_params)

if get_run_captures_response.ok:
    run_captures_json = get_run_captures_response.json()
    run_captures = run_captures_json.get('resources')
    for capture in run_captures:
        time = capture.get('time')

        key_results = capture.get('keyResults')
        max_temp = key_results[0].get('value')
        min_temp = key_results[1].get('value')
        
        print(f'{time}\t{max_temp}\t{min_temp}')
else:
    print('get_run_captures() failed')
    print(get_run_captures_response.text)

In [ ]:
room_heater_params = {
    'missionName': MISSION,
    'actionName': ROOM,
    'channelName': CHANNEL
}
get_run_captures_response = scout_client.get_run_captures(params=room_heater_params)

if get_run_captures_response.ok:
    run_captures_json = get_run_captures_response.json()
    run_captures = run_captures_json.get('resources')
    for capture in run_captures:
        time = capture.get('time')

        key_results = capture.get('keyResults')
        max_temp = key_results[0].get('value')
        min_temp = key_results[1].get('value')
        
        print(f'{time}\t{max_temp}\t{min_temp}')
else:
    print('get_run_captures() failed')
    print(get_run_captures_response.text)

### Task 4

In [ ]:
def get_thermal_data(action):
    data = []

    res = scout_client.get_run_captures(params={
        'missionName': MISSION,
        'actionName': action,
        'channelName': CHANNEL
    })

    if res.ok:
        captures = res.json().get('resources')
        for capture in captures:
            time = capture.get('time')
            key_results = capture.get('keyResults')
            max_temp = key_results[0].get('value')
            min_temp = key_results[1].get('value')
            data.append([time, max_temp, min_temp])
    else:
        print('get_run_captures() failed')
        print(get_run_captures_response.text)
        
    return data


def write_locally(data, filename):
    with open(f'{filename}', 'w') as out:
        for line in data:
            out.write(f'{line[0]},{line[1]},{line[2]}\n')

In [ ]:
def upload_to_aws(source_filename, bucket_name, destination_filename):
    s3 = boto3.client('s3')
    s3.upload_file(source_filename, bucket_name, destination_filename)

In [ ]:
NAME = 'ShamusClifford'

wall_data = get_thermal_data(WALL)
wall_filename = f'{DAY}-{WALL}-{NAME}.csv'
room_data = get_thermal_data(ROOM)
room_filename = f'{DAY}-{ROOM}-{NAME}.csv'

write_locally(wall_data, wall_filename)
write_locally(room_data,  room_filename)

upload_to_aws(source_filename=wall_filename, bucket_name=BUCKET_NAME, destination_filename=f'{wall_filename}')
upload_to_aws(source_filename=room_filename, bucket_name=BUCKET_NAME, destination_filename=f'{room_filename}')